
## Ejercicio 3

Para resolver este ejercicio utilice un modelo de red neuronal convolucional que reconozca la cantidad de dedos extendidos en cada mano de las imágenes del dataset **“Fingers”**.

<div style="text-align: center;">
    <img src='../../images/p4-ej10.png' width="40%">
</div>

La versión original de este de estas imágenes se encuentra en https://www.kaggle.com/koryakinp/fingers.

Puede hallar una versión reducida de estas imágenes en el Moodle del curso, en la misma sección donde se encuentra este enunciado de práctica. También encontrará allí ejemplos sobre cómo cargar estas imágenes y cómo procesarlas con una red neuronal convolucional.

### a)

Entrene y pruebe un modelo utilizando los datos de las carpetas **test** y **train**, midiendo **accuracy**.

In [5]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

# Definir rutas
TRAIN_DIR = '../../Datos/fingers/train'
TEST_DIR = '../../Datos/fingers/test'

# Parámetros
IMG_SIZE = (64, 64)  # Ajusta según tu dataset
BATCH_SIZE = 32

# Cargar datasets automáticamente
train_dataset = tf.keras.utils.image_dataset_from_directory(
    TRAIN_DIR,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode='categorical'  # Para clasificación multiclase
)

val_dataset = tf.keras.utils.image_dataset_from_directory(
    TRAIN_DIR,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode='categorical'
)

test_dataset = tf.keras.utils.image_dataset_from_directory(
    TEST_DIR,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode='categorical',
    shuffle=False  # Para mantener orden en test
)

# ¡OBTENER CLASES ANTES DE NORMALIZAR!
class_names = train_dataset.class_names
print("Clases encontradas:", class_names)
print("Número de clases:", len(class_names))

# Normalizar imágenes (0-1) DESPUÉS de obtener class_names
normalization_layer = tf.keras.layers.Rescaling(1./255)
train_dataset = train_dataset.map(lambda x, y: (normalization_layer(x), y))
val_dataset = val_dataset.map(lambda x, y: (normalization_layer(x), y))
test_dataset = test_dataset.map(lambda x, y: (normalization_layer(x), y))

Found 18000 files belonging to 6 classes.
Using 14400 files for training.
Found 18000 files belonging to 6 classes.
Using 3600 files for validation.
Found 3600 files belonging to 6 classes.
Clases encontradas: ['0', '1', '2', '3', '4', '5']
Número de clases: 6


### b)

Genere una versión del **dataset** para **test** agregando transformaciones al azar sobre imágenes originales. Haga **rotaciones** entre -45° y 45°, repita el test y mida el accuracy.

### c)

Genere una versión del **dataset train** como en b) y repita el entrenamiento y prueba del punto a) con los datasets modificados.